<a href="https://colab.research.google.com/github/manuelhedez/codigos-python/blob/main/Felicitaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -q streamlit

In [19]:
%%writefile app.py
import streamlit as st
import streamlit as st
import csv
import random
from datetime import datetime, timedelta, date
import os
import smtplib
from email.mime.text import MIMEText
# === CLASES ===
class cumpleaños:
    def __init__(self, nombre, año, mes, dia, correo):
        self._nombre = nombre
        self.fecha = date(año, mes, dia)
        self.correo = correo

    @property
    def fecha(self):
        return self._fecha
    @fecha.setter
    def fecha(self, fecha):
        self._fecha = fecha

    @property
    def nombre(self):
        return self._nombre
    @nombre.setter
    def nombre(self, nuevonombre):
        self._nombre = nuevonombre

    @property
    def correo(self):
        return self._correo
    @correo.setter
    def correo(self, correo):
        self._correo = correo

class mensaje:
    def __init__(self, mensaje):
        self._mensaje = mensaje
    @property
    def mensaje(self):
        return self._mensaje
    @mensaje.setter
    def mensaje(self, mensaje):
        self._mensaje = mensaje

# === FUNCIONES AUXILIARES ===
def calcular_cuenta_regresiva(fecha_nacimiento):
    ahora = datetime.now()
    hoy = ahora.date()
    proximo_cumple = fecha_nacimiento.replace(year=hoy.year)
    if proximo_cumple < hoy:
        proximo_cumple = proximo_cumple.replace(year=hoy.year + 1)
    proximo_cumple_dt = datetime.combine(proximo_cumple, datetime.min.time())
    diferencia = proximo_cumple_dt - ahora
    return diferencia

def cargar_personas():
    personas = []
    try:
        with open("datoscump.csv", mode="r", newline="", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                nombre = row["Nombre"]
                fecha_str = row["fecha"]
                correo = row["correo"]
                año, mes, dia = map(int, fecha_str.split("-"))
                personas.append(cumpleaños(nombre, año, mes, dia, correo))
    except FileNotFoundError:
        pass
    return personas

def cargar_mensajes():
    mensajes = []
    try:
        with open("mensaje.csv", mode="r", newline="", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                mensajes.append(mensaje(row["mensaje"]))
    except FileNotFoundError:
        pass
    return mensajes
def enviar_felicitacion(destinatario, nombre, mensaje_texto):
    remitente = "manuel.hernandez9742@alumnos.udg.mx"
    contraseña = "upru phlt tcfk aqfq"

    msg = MIMEText(f"¡Feliz cumpleaños, {nombre}!\n\n{mensaje_texto}")
    msg["Subject"] = "🎉 ¡Feliz cumpleaños!"
    msg["From"] = remitente
    msg["To"] = destinatario

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(remitente, contraseña)
        server.send_message(msg)

# === INTERFAZ STREAMLIT ===
st.set_page_config(page_title="Gestor de Cumpleaños 🎂", layout="centered")
st.title("🎉 Gestor de Cumpleaños con Mensajes")

personas = cargar_personas()
mensajes = cargar_mensajes()

st.subheader("📋 Lista de Cumpleaños")
if personas:
    for persona in personas:
        cuenta = calcular_cuenta_regresiva(persona.fecha)
        dias, segundos = cuenta.days, cuenta.seconds
        horas = segundos // 3600
        minutos = (segundos % 3600) // 60
        segundos = segundos % 60
        st.write(f"**{persona.nombre}** | {persona.fecha} | Faltan `{dias}d {horas}h {minutos}m`")
else:
    st.info("No hay cumpleaños registrados.")

# === FORMULARIOS ===
with st.expander("➕ Agregar cumpleaños"):
    with st.form("form_cumple"):
        nombre = st.text_input("Nombre")
        fecha = st.date_input("Fecha de nacimiento")
        correo = st.text_input("Correo")
        enviar = st.form_submit_button("Guardar cumpleaños")
        if enviar:
            nuevo = cumpleaños(nombre, fecha.year, fecha.month, fecha.day, correo)
            with open("datoscump.csv", mode="a", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                if os.stat("datoscump.csv").st_size == 0:
                    writer.writerow(["Nombre", "fecha", "correo"])
                writer.writerow([nuevo.nombre, nuevo.fecha, nuevo.correo])
            st.success(f"Se agregó a {nuevo.nombre}")

with st.expander("💬 Agregar mensaje"):
    with st.form("form_mensaje"):
        texto = st.text_area("Mensaje de felicitación")
        enviar = st.form_submit_button("Guardar mensaje")
        if enviar:
            nuevo = mensaje(texto)
            with open("mensaje.csv", mode="a", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                if os.stat("mensaje.csv").st_size == 0:
                    writer.writerow(["mensaje"])
                writer.writerow([nuevo.mensaje])
            st.success("Mensaje guardado")

# === ENVÍO AUTOMÁTICO (opcional en botón) ===
with st.expander("📬 Revisar cumpleaños y enviar mensajes"):
    hoy = date.today()
    cumple_hoy = [p for p in personas if p.fecha.day == hoy.day and p.fecha.month == hoy.month]

    if cumple_hoy and mensajes:
        st.write(f"🎉 Hay {len(cumple_hoy)} cumpleaños hoy.")

        opcion = st.radio(
            "¿Quieres elegir un mensaje o enviar uno aleatorio?",
            ("Aleatorio", "Elegir mensaje específico"),
            key="tipo_mensaje"
        )

        opciones_mensajes = [m.mensaje for m in mensajes]
        if opcion == "Elegir mensaje específico":
            mensaje_seleccionado = st.selectbox("Selecciona el mensaje a enviar", opciones_mensajes, key="mensaje_manual")
        else:
            mensaje_seleccionado = random.choice(opciones_mensajes)

        if st.button("Enviar mensajes"):
            for persona in cumple_hoy:
                try:
                    enviar_felicitacion(persona.correo, persona.nombre, mensaje_seleccionado)
                    st.success(f"Mensaje enviado a {persona.nombre}")
                except Exception as e:
                    st.error(f"Error al enviar a {persona.nombre}: {e}")
    else:
        st.warning("No hay cumpleaños hoy o no hay mensajes cargados.")


Overwriting app.py


In [20]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 760ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [21]:
!streamlit run /content/app.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.16.138.158


In [22]:
!npx localtunnel --port 8501

⠙your url is: https://tender-pugs-cough.loca.lt
^C
